# Complete Guide: Flask App Deployment with CI/CD to Azure

This comprehensive guide walks you through creating a Python Flask application, containerizing it with Docker, setting up CI/CD with GitHub Actions, and deploying to Azure App Service.

## Table of Contents

1. [Prerequisites](#prerequisites)
2. [Step 1: Create Flask Application](#step-1-create-flask-application)
3. [Step 2: Set Up Project Structure](#step-2-set-up-project-structure)
4. [Step 3: Containerize with Docker](#step-3-containerize-with-docker)
5. [Step 4: Set Up GitHub Repository](#step-4-set-up-github-repository)
6. [Step 5: Configure GitHub Actions CI/CD](#step-5-configure-github-actions-cicd)
7. [Step 6: Deploy to Azure App Service](#step-6-deploy-to-azure-app-service)
8. [Step 7: Set Up Continuous Deployment](#step-7-set-up-continuous-deployment)
9. [Step 8: Test the Deployment](#step-8-test-the-deployment)
10. [Troubleshooting](#troubleshooting)
11. [Next Steps](#next-steps)


## Prerequisites

Before starting, ensure you have the following installed:

- **Python 3.11+** - [Download Python](https://www.python.org/downloads/)
- **Git** - [Download Git](https://git-scm.com/downloads)
- **Docker Desktop** - [Download Docker](https://www.docker.com/products/docker-desktop/)
- **Azure CLI** - [Install Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli)
- **GitHub Account** - [Sign up for GitHub](https://github.com/)
- **Azure Account** - [Sign up for Azure](https://azure.microsoft.com/free/)


## Step 1: Create Flask Application

### 1.1 Create Project Directory


In [ ]:
mkdir flask-app
cd flask-app


### 1.2 Create Virtual Environment

**Windows (PowerShell):**
```powershell
python -m venv venv
.\venv\Scripts\Activate.ps1
```

**Windows (CMD):**
```cmd
python -m venv venv
venv\Scripts\activate
```

**Linux/Mac:**
```bash
python3 -m venv venv
source venv/bin/activate
```


### 1.3 Create Flask Application

Create a file named `main.py`:


In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({
        'message': 'Hello, World!',
        'status': 'success'
    })

@app.route('/health')
def health():
    return jsonify({'status': 'healthy'}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)


### 1.4 Create Requirements File

Create `requirements.txt`:


In [ ]:
Flask==3.0.0
gunicorn==21.2.0


### 1.5 Install Dependencies


In [ ]:
pip install -r requirements.txt


### 1.6 Test Locally

Run `python main.py` and visit `http://localhost:5000` in your browser.

You should see:
```json
{"message":"Hello, World!","status":"success"}
```

Visit `http://localhost:5000/health` to see:
```json
{"status":"healthy"}
```


## Step 2: Set Up Project Structure

### 2.1 Create .gitignore


In [ ]:
# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/
venv/
ENV/
.venv

# Flask
instance/
.webassets-cache

# Environment variables
.env
.env.local

# IDE
.vscode/
.idea/
*.swp
*.swo

# Testing
.pytest_cache/
.coverage
htmlcov/

# Logs
*.log

# OS
.DS_Store
Thumbs.db


## Step 3: Containerize with Docker

### 3.1 Create Dockerfile


In [ ]:
# Use Python 3.11 slim image
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Copy requirements file
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY main.py .

# Expose port 5000
EXPOSE 5000

# Use gunicorn to run the Flask app
CMD ["gunicorn", "--bind", "0.0.0.0:5000", "--workers", "4", "main:app"]


### 3.2 Build and Test Docker Image


In [ ]:
# Build Docker image
docker build -t flask-app .

# Run container
docker run -p 5000:5000 flask-app


## Step 4: Set Up GitHub Repository

### 4.1 Initialize Git and Push to GitHub


In [ ]:
# Configure Git (first time only)
git config --global user.name "Your Name"
git config --global user.email "your.email@example.com"

# Initialize repository
git init
git add .
git commit -m "Initial Flask app with Docker setup"

# Add remote (replace with your GitHub username)
git remote add origin https://github.com/YOUR_USERNAME/flask-app.git
git branch -M main
git push -u origin main


## Step 5: Configure GitHub Actions CI/CD

### 5.1 Create Workflow File

Create `.github/workflows/deploy.yml` with the CI/CD pipeline configuration.

The workflow will:
- Run tests on every push
- Build Docker image
- Deploy to Azure (on main branch)


## Step 6: Deploy to Azure App Service

### 6.1 Login to Azure


In [ ]:
az login


### 6.2 Create Resource Group and App Service


In [ ]:
# Create resource group
az group create --name flask-rg --location eastus

# Create app service plan
az appservice plan create --name flask-plan --resource-group flask-rg --sku B1 --is-linux

# Create web app (replace YOUR_APP_NAME with unique name)
az webapp create --resource-group flask-rg --plan flask-plan --name YOUR_APP_NAME --runtime "PYTHON:3.11"


### 6.3 Configure Startup Command

In Azure Portal:
1. Go to your App Service
2. Configuration → General settings
3. Startup Command: `gunicorn --bind=0.0.0.0:8000 main:app`
4. Save

**Note:** Azure uses port 8000, not 5000!


## Step 7: Set Up Continuous Deployment

### 7.1 Create Azure Service Principal


In [ ]:
# Get subscription ID
az account show --query id -o tsv

# Get resource group name
az group list --query "[].name" -o tsv

# Create service principal (replace with your values)
az ad sp create-for-rbac --name "github-actions-flask-deploy" \
  --role contributor \
  --scopes /subscriptions/<SUBSCRIPTION_ID>/resourceGroups/<RESOURCE_GROUP>/providers/Microsoft.Web/sites/<APP_NAME> \
  --sdk-auth


### 7.2 Add Secret to GitHub

1. GitHub → Settings → Secrets and variables → Actions
2. New repository secret
3. Name: `AZURE_CREDENTIALS`
4. Value: Paste the JSON from step 7.1
5. Add secret


### 7.3 Update Workflow for Azure Deployment

Update `.github/workflows/main_flask-app1.yml` to use `AZURE_CREDENTIALS` secret and deploy to Azure.

Key changes:
- Use `creds: ${{ secrets.AZURE_CREDENTIALS }}` for Azure login
- Add `startup-command: 'gunicorn --bind=0.0.0.0:8000 main:app'`
- Set correct app name


## Step 8: Test the Deployment

### 8.1 Make a Test Change

Update `main.py` message and push:

```python
'message': 'Hello from Azure! Automatic deployment works!'
```

### 8.2 Commit and Push

```bash
git add main.py
git commit -m "Test automatic deployment"
git push
```

### 8.3 Verify

1. Check GitHub Actions - workflow should run automatically
2. Wait 1-2 minutes for deployment
3. Visit your Azure app URL
4. See the updated message!

**Congratulations! Your CI/CD pipeline is fully automated! 🎉**


## Troubleshooting

### Common Issues:

1. **Docker build fails**
   - Check Dockerfile syntax
   - Verify all files exist

2. **Azure login fails**
   - Use `az login --use-device-code`

3. **App shows default Azure page**
   - Set startup command in Azure Portal

4. **Port errors**
   - Azure uses port 8000, not 5000

5. **GitHub Actions deployment fails**
   - Verify `AZURE_CREDENTIALS` secret is set correctly
   - Check JSON format


## Summary

You've successfully:

✅ Created a Flask application  
✅ Containerized with Docker  
✅ Set up GitHub repository  
✅ Configured CI/CD with GitHub Actions  
✅ Deployed to Azure App Service  
✅ Set up continuous deployment  
✅ Tested automatic deployment  

Your Flask app now has a complete CI/CD pipeline that automatically builds, tests, and deploys on every code push!

## Resources

- [Flask Documentation](https://flask.palletsprojects.com/)
- [Docker Documentation](https://docs.docker.com/)
- [GitHub Actions Documentation](https://docs.github.com/en/actions)
- [Azure App Service Documentation](https://docs.microsoft.com/en-us/azure/app-service/)

**Happy Coding! 🚀**
